<a href="https://colab.research.google.com/github/morioka/LlamaIndex_trial/blob/main/llmaindex_question_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LlamaIndexを用いて、クイズ生成を考える

OpenAI API (GPT-3, ChatGPT, GPT-4)に対して、適切なプロンプトを設定して、クイズを作成し、そのよさを検証することを試みてみます。

ここでは OpenAI APIをそのまま用いるのではなく、LlamaIndex (旧GPT-index)というフレームワークを用います。参考にした記事は、次のとおり：
- [ChatGPTで独自データを利用できるLlamaIndexはどんな仕組みで動いているのか？調べてみました | DevelopersIO](https://dev.classmethod.jp/articles/llamaindex-overview/)


ある記事を選択し、記事中の特徴的な表現（固有表現）を正解として、その記事に基づいたクイズを生成することを目指します。また、クイズ問題文と正解の対応が適切か、クイズ問題文の難易度や面白さを評価させてみます。


以降の流れは以下のとおり:
- 記事本文を取得する
- 解答候補を抽出する
- 関連情報を得る
- インデックスを作成する
- 質問生成：四択問題
- 検証: 質問＋各選択肢が成立するか
- 導入文の生成
- 検証: ハルシネーションのチェック
- 検証: 導入文と質問・解答をペアにしてすわりがよいかを評価
- 検証: クイズの難しさの評価
- 検証: クイズとしての面白さの評価







In [ ]:
!pip install -qU pip wheel setuptools

## 対象の記事とその本文を取得する

ここでは [kyodo_news](https://nordot.app/-/units/39166665832988672) に掲載されている記事の中から一つ選ぶことにします。

In [ ]:
# この例では、この記事を使います。(すでに使えない可能性があります)

article_urls=['https://nordot.app/1014472568104288256']  # 細野勇策が通算16アンダー首位　東建男子ゴルフ第2日 ｜ 共同通信 (2023/03/31)

In [ ]:
# nordotから一覧する場合の例は、以下のとおりです。

from urllib import request  # urllib.requestモジュールをインポート
import json

url = 'https://nordot.app/-/units/39166665832988672/list?offset=10&limit=10'
user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'   # User-Agentを設定しないと応答を返さない
headers = {'User-Agent': user_agent}

req = request.Request(url, headers=headers)
with request.urlopen(req) as response:
   content = json.loads(response.read().decode('utf8'))

[
    (p["title"], p["published_at"], f'https://nordot.app/{p["id"]}') for p in content['posts']
]


[('ウクライナ問題、対話が唯一の道',
  '2023-04-14T13:17:05+00:00',
  'https://nordot.app/1019592487259455488'),
 ('藤波朱理「パリ五輪で優勝を」',
  '2023-04-14T13:07:27+00:00',
  'https://nordot.app/1019594318025064448'),
 ('3月の米小売売上高1.0％減',
  '2023-04-14T13:06:41+00:00',
  'https://nordot.app/1019595132064350208'),
 ('NY円、132円台後半',
  '2023-04-14T12:58:47+00:00',
  'https://nordot.app/1019593267427508224'),
 ('中9―2巨（14日）',
  '2023-04-14T12:57:38+00:00',
  'https://nordot.app/1019586499306536960'),
 ('木星と衛星の探査機打ち上げ',
  '2023-04-14T12:54:19+00:00',
  'https://nordot.app/1019591731507757056'),
 ('佐々木、首位でストリート決勝へ',
  '2023-04-14T12:54:02+00:00',
  'https://nordot.app/1019575124810694656'),
 ('中国外相、ロシアに武器提供せず',
  '2023-04-14T12:53:46+00:00',
  'https://nordot.app/1019591232043892736'),
 ('楽3―0ソ（14日）',
  '2023-04-14T12:53:23+00:00',
  'https://nordot.app/1019586268548218880'),
 ('D8―3神（14日）',
  '2023-04-14T12:51:55+00:00',
  'https://nordot.app/1019587209956376576')]

## BeautifulSoupをつかって記事本文を抽出

Webページそのままを、SimpleWebPageReaderで読み込んでインデックスを作成する場合、HTMLほかのタグも含めてインデックスしてしまいます。

BeautifulSoupを使って記事本文をスクレイピングしてインデックスすると、きれいな情報を取得でき、それを使った結果もよさそうです。

※記事によっては、写真に対するキャプションの削除や、カッコ表記の削除または展開などの前処理が必要になるかもしれません。

In [ ]:
!pip install -qU bs4

  Preparing metadata (setup.py) ... done


In [ ]:
from urllib import request
from bs4 import BeautifulSoup

text = ''

if False:  # 何度も取得することははばかられるので、取得済のものを用います。
  url = article_urls[0]
  response = request.urlopen(url)
  soup = BeautifulSoup(response)
  response.close()

  text = soup.find('div', class_='main__articleBody').text.strip().replace('\u3000', ' ')

else:
  # 何度も取得することははばかられるので、記事本文を用意します。
  text = '第2日、12番でアプローチショットを放つ細野勇策。通算16アンダーで首位＝東建多度CC名古屋 東建ホームメイト・カップ第2日（31日・三重県東建多度CC名古屋＝7062ヤード、パー71）20歳でツアー未勝利の細野勇策が3位から11バーディー、1ボギーの61をマークし、通算16アンダー、126で首位に立った。61は同コース開催での大会最少スコアを1打更新。4打差で前日首位の今平周吾が続いた。通算10アンダーの3位に64で回った田中裕基とマイケル・ヘンドリー（ニュージーランド）がつけ、さらに1打差で石川遼らが並んだ。金谷拓実は首位から71と伸ばせず7アンダーの12位に後退。昨年優勝の香妻陣一朗は3アンダーの50位。2アンダーまでの79人が決勝ラウンドに進んだ。'

text  

'第2日、12番でアプローチショットを放つ細野勇策。通算16アンダーで首位＝東建多度CC名古屋 東建ホームメイト・カップ第2日（31日・三重県東建多度CC名古屋＝7062ヤード、パー71）20歳でツアー未勝利の細野勇策が3位から11バーディー、1ボギーの61をマークし、通算16アンダー、126で首位に立った。61は同コース開催での大会最少スコアを1打更新。4打差で前日首位の今平周吾が続いた。通算10アンダーの3位に64で回った田中裕基とマイケル・ヘンドリー（ニュージーランド）がつけ、さらに1打差で石川遼らが並んだ。金谷拓実は首位から71と伸ばせず7アンダーの12位に後退。昨年優勝の香妻陣一朗は3アンダーの50位。2アンダーまでの79人が決勝ラウンドに進んだ。'

## 解答候補を抽出する

ここでの解答候補は、spacy/GiNZAを用いて固有表現抽出した結果とします。

In [ ]:
!pip install -qU spacy ja_ginza

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 53.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 52.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.5.0 requires spacy<3.6.0,>=3.5.0, but you have spacy 3.4.4 which is incompatible.


In [ ]:
import spacy

nlp=spacy.load('ja_ginza')

In [ ]:
if False:
  doc = nlp(text)
  # 記事本文を対象にすると名字が頻出し、別人の関連情報を引っ張ってきやすくなるようです。
else:
  #summary_text = index.query("要約してください").response.strip()
  summary_text = '細野勇策が3位から11バーディー、1ボギーの61で通算16アンダー、126で首位に立ち、大会最少スコアを1打更新した。田中裕基とマイケル・ヘンドリーが3位につけ、石川遼らが1打差で並んだ。金谷拓実は7アンダーの12位に後退し、香妻陣一朗は3アンダーの50位であった。79人が2アンダーまでで決勝ラウンドに進んだ。'
  doc = nlp(summary_text)

doc

細野勇策が3位から11バーディー、1ボギーの61で通算16アンダー、126で首位に立ち、大会最少スコアを1打更新した。田中裕基とマイケル・ヘンドリーが3位につけ、石川遼らが1打差で並んだ。金谷拓実は7アンダーの12位に後退し、香妻陣一朗は3アンダーの50位であった。79人が2アンダーまでで決勝ラウンドに進んだ。

In [ ]:
[ (e.text, e.label_) for e in doc.ents ]

[('細野勇策', 'Person'),
 ('3位', 'Rank'),
 ('11バーディー', 'N_Organization'),
 ('1ボギー', 'Period_Day'),
 ('61', 'Numex_Other'),
 ('16アンダー', 'N_Product'),
 ('126', 'Numex_Other'),
 ('首位', 'Rank'),
 ('1打', 'N_Event'),
 ('田中裕基', 'Person'),
 ('マイケル・ヘンドリー', 'Person'),
 ('3位', 'Rank'),
 ('石川遼', 'Person'),
 ('1打', 'N_Event'),
 ('金谷拓実', 'Person'),
 ('7アンダー', 'N_Product'),
 ('12位', 'Rank'),
 ('香妻', 'Person'),
 ('3アンダー', 'N_Product'),
 ('50位', 'Rank'),
 ('79人', 'N_Person'),
 ('2アンダー', 'N_Product')]

## 関連情報を得る

解答候補やその他のキーワードに対して関連する情報を合わせて作問できると、良いものになると考えます。

ここでは、ありがちですがひとまず Wikipedia 日本語版の記事サマリーを用います。

ある解答候補について関連情報を得たら、そちらに基づいて質問を生成してもよい。

- [PythonでWikipediaの情報を取得する | 分析ノート](https://analytics-note.xyz/programming/python-wikipedia/)
- [wikipedia · PyPI](https://pypi.org/project/wikipedia/)

In [ ]:
!pip install -qU wikipedia

  Preparing metadata (setup.py) ... done


In [ ]:
import wikipedia
wikipedia.set_lang("ja")

In [ ]:
for e in doc.ents:
  if e.label_ in ['Person', 'Company', 'Country']:
    print( e.text,  wikipedia.search(e.text, results=3))

細野勇策 ['日本の写真家一覧', 'さすがの猿飛', '岐阜県立岐阜高等学校']
田中裕基 ['Over The Top (バンド)', '田中裕子', '田中裕二 (お笑い芸人)']
マイケル・ヘンドリー ['マイケル・ジャクソン', 'ジミ・ヘンドリックス', 'マイケル・ランドウ']
石川遼 ['石川遼', '石川遼スペシャル RESPECT 〜ゴルフを愛する人々へ〜', 'ゴルフ日本シリーズ']
金谷拓実 ['金谷拓実', 'SOMPOひまわり生命保険', 'インターナショナル・マネジメント・グループ']
香妻 ['香妻琴乃', '香妻陣一朗', 'ゴルファー一覧']


「さすがの猿飛」は細野不二彦。おそらく間違い。

In [ ]:
# 解答候補でwikipediaを検索。解答候補が含まれるページsummaryを列挙
# ページsummaryでは情報不足か。
# しかし、ページcontentでは分量過多を懸念
rel_urls = []
for e in doc.ents:
  if e.label_ in ['Person', 'Company', 'Country']:
    wp_ents = wikipedia.search(e.text, results=3)
    for wp in wp_ents:
      try:
        wp_page = wikipedia.page(wp)
        if e.text in wp_page.summary:
          rel_urls.append(wp_page.summary)
      except:
        pass

rel_urls

['『石川遼スペシャル RESPECT 〜ゴルフを愛する人々へ〜』（いしかわりょうスペシャル リスペクト ゴルフをあいするひとびとへ）は、2010年4月4日から2012年3月31日まで、テレビ東京系列で放送されていたゴルフトーク番組。石川遼の冠番組。\n\n',
 '香妻 琴乃（こうづま ことの、1992年4月17日 - ）は、日本の女子プロゴルファー。鹿児島県鹿屋市出身。サマンサタバサ所属。',
 '香妻 陣一郎（こうづま じんいちろう、1994年7月7日 - ）は、日本のプロゴルファー。鹿児島県鹿屋市出身、身長165cm、日章学園高校卒業、国際スポーツ振興協会所属。妻はモデルの武井玲奈。姉は女子プロゴルファーの香妻琴乃。']

適切な関連情報を得ることは重要。ja_ginza_electraの場合、姓だけ名だけでも拾い上げてしまい、その結果、無関係なwikipedia記事を関連あるものと誤って判断することがある。一概に強力なモデルであればよいとは言えない。

上記を見るかぎりは、wikipedia summaryの品質は高くない。全文を取得して要約させたほうがよいかもしれない。

wikidataからグラフ表現を引っ張ってくるのもよいかもしれない。

## デバッグログ設定

In [ ]:
# デバグログが必要な場合は、以下のlogging.basicConfig設定を有効にします。

import logging
import sys
#logging.basicConfig(stream=sys.stdout, level=logging.DEBUG, force=True)

----

## ここからLlamaIndex (~ GPT API)を利用

## インデックスを作成する

- 前述のように、BeautifulSoupでスクレイピングした記事本文を使います


In [ ]:
!pip install -qU llama-index wikipedia langchain html2text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.7/172.7 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.7/511.7 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 4.8 MB/s eta 0:00:00


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = 'YOUR_OPENAI_API_KEY'

In [ ]:
from llama_index import GPTSimpleVectorIndex, StringIterableReader

documents = StringIterableReader().load_data([text])
index = GPTSimpleVectorIndex.from_documents(documents)

↑上記でLLMを指定しないと、GPT-4が使われるかもしれません。高単価

In [ ]:
index.query("<question_text>?")

Response(response='\n細野勇策は、第2日の試合で首位に立った。', source_nodes=[NodeWithScore(node=Node(text='第2日、12番でアプローチショットを放つ細野勇策。通算16アンダーで首位＝東建多度CC名古屋 東建ホームメイト・カップ第2日（31日・三重県東建多度CC名古屋＝7062ヤード、パー71）20歳でツアー未勝利の細野勇策が3位から11バーディー、1ボギーの61をマークし、通算16アンダー、126で首位に立った。61は同コース開催での大会最少スコアを1打更新。4打差で前日首位の今平周吾が続いた。通算10アンダーの3位に64で回った田中裕基とマイケル・ヘンドリー（ニュージーランド）がつけ、さらに1打差で石川遼らが並んだ。金谷拓実は首位から71と伸ばせず7アンダーの12位に後退。昨年優勝の香妻陣一朗は3アンダーの50位。2アンダーまでの79人が決勝ラウンドに進んだ。', doc_id='6b9ef732-3e07-45af-a9bd-37fbb0614fd9', embedding=None, doc_hash='e08cc6f12c10350e3c9bd25bcc18f9b7ff2b9d1a14acb1ea9e37f3ba6efb87fc', extra_info=None, node_info={'start': 0, 'end': 332}, relationships={<DocumentRelationship.SOURCE: '1'>: '232bfa27-8dc5-4076-9798-d228459b00e9'}), score=0.6848365464618824)], extra_info=None)

In [ ]:
index.query("次の問いに日本語で答えてください。単語で出力してください。\n\n誰が首位ですか?").response.strip()

'細野勇策'

In [ ]:
index.query("要約してください").response.strip()

'20歳の細野勇策が三重県東建多度CC名古屋で開催された東建ホームメイト・カップ第2日で11バーディー、1ボギーの61をマークし、通算16アンダー、126で首位に立ちました。今平周吾が4打差で2位、田中裕基とマイケル・ヘンドリー（ニュージーランド）が3位、石川遼らが1打差で4位となりました。金谷拓実は7アンダーの12位、昨年優勝の香妻陣一朗は3アンダーの50位でした。79人が2アンダー'

## 一度保存

インデックスを作成するたびにOPENAI APIのトークンを消費するので、できるだけ保存して、それを使うのがよいでしょう。



In [ ]:
index.save_to_disk('index.json')

## 関連情報をインデックス

In [ ]:
if False:
  documents_rel = SimpleWebPageReader().load_data(rel_urls)
  index_rel = GPTSimpleVectorIndex.from_documents(documents_rel)

In [ ]:
from llama_index import StringIterableReader, GPTSimpleVectorIndex

documents_rel = StringIterableReader().load_data(rel_urls)
index_rel = GPTSimpleVectorIndex.from_documents(documents_rel)

In [ ]:
index_rel.save_to_disk('index_rel.json')

## 質問生成 四択問題

記事をインデックスしたものに対して、クイズ生成を指示します。


関連情報をインデックスしたものに対して、記事を提示してクイズ生成を指示するほうがよいかもしれません。

In [ ]:
index.query("あなたは有能なクイズ作家です。細野勇作が正解になる四択クイズを作ってください。").response.strip()  # 実は名前を間違い

'Q. 細野勇策が第2日に放ったアプローチショットで、通算16アンダーで首位に立ったのはどこで開催された大会でしたか？\n\nA. \nA. 東建多度CC名古屋\nB. 東京都ゴルフクラブ\nC. 東海ゴルフクラブ\nD. 東京都ゴルフ倶楽部'

In [ ]:
index.query("あなたは有能なクイズ作家です。細野勇策が正解になる四択クイズを作ってください。").response.strip() 

'Q. 細野勇策は、東建多度CC名古屋で開催された東建ホームメイト・カップ第2日において、何スコアで首位に立ったか？\n\nA. \nA. 61\nB. 64\nC. 71\nD. 126'

In [ ]:
index.query("あなたは有能なクイズ作家です。細野勇策が正解になる四択クイズを作ってください。3つ作ってください").response.strip()

'1. 細野勇策は、東建多度CC名古屋で開催された東建ホームメイト・カップ第2日で何スコアをマークしましたか？\nA. 59\nB. 61\nC. 63\nD. 65\n\n2. 細野勇策は、東建多度CC名古屋で開催された東建ホームメイト・カップ第2日で何位になりましたか？\nA. 1位\nB. 2位\nC. 3位\nD. 4位\n\n3. 東建多度CC名古屋で開催された東建ホームメイト・カップ第2日で、細野勇'

生成内容の指定が不十分。細野勇作を正解とする質問を生成するよう指示したが、得られたものいずれも細野勇作についての質問であって、細野勇作が正解になる質問ではない。

最後の指示では、出力が途中で切られている。大きめの出力長の指定が必要。

## 関連記事を前提として質問生成

関連記事を前提として = インデックスとして、
それに対して注目記事を条件付けとして与えます。

あるドメインについての知識があって、それを前提に作問することを考えると、こちらのほうが自然な流れかもしれません。

In [ ]:
index_rel.query("""あなたは有能なクイズ作家です。以下の記事に基づいて、細野勇作が正解になる四択クイズを作ってください。

記事: 第2日、12番でアプローチショットを放つ細野勇策。通算16アンダーで首位＝東建多度CC名古屋 東建ホームメイト・カップ第2日（31日・三重県東建多度CC名古屋＝7062ヤード、パー71）20歳でツアー未勝利の細野勇策が3位から11バーディー、1ボギーの61をマークし、通算16アンダー、126で首位に立った。61は同コース開催での大会最少スコアを1打更新。4打差で前日首位の今平周吾が続いた。通算10アンダーの3位に64で回った田中裕基とマイケル・ヘンドリー（ニュージーランド）がつけ、さらに1打差で石川遼らが並んだ。金谷拓実は首位から71と伸ばせず7アンダーの12位に後退。昨年優勝の香妻陣一朗は3アンダーの50位。2アンダーまでの79人が決勝ラウンドに進んだ。

""").response.strip()  # 実は名前を間違い

'四択クイズ: 細野勇策が第2日に放つアプローチショットで通算16アンダーをマークし、首位に立ったのはどこの大会でしたか？\n\nA. 東建多度CC名古屋\nB. 東建ホームメイト・カップ\nC. ニュージーランド\nD. 鹿児島県鹿屋市'

ただし、この例では関連情報が活きたものではなさそう。

## 検証: 質問＋各選択肢が成立するか

正例負例それぞれで成立不成立を問い直すことで、生成した質問の妥当性を測ることにします。

判断の根拠を示させることも有効だろう。

In [ ]:
query = """あなたは有能なクイズ作家です。次の質問と回答のペアは適切ですか。YES/NOで答えてください。

質問: 細野勇策が通算16アンダー首位となったのはどこで行われたゴルフ大会でしたか？
回答: 東建ホームメイト・カップ
"""

In [ ]:
index.query(query).response.strip()

'Yes'

In [ ]:
query_template = """あなたは有能なクイズ作家です。次の質問と回答のペアは適切ですか。YES/NOで答えてください。


質問: {question}
回答: {answer}
"""

In [ ]:
question = "第2日の大会で首位に立ったのは誰でしたか？"
answer = "今平周吾"
#answer = "細野勇策"
#answer = "田中裕基"
#answer = "マイケル・ヘンドリー"

query = query_template.format(question=question, answer=answer)
index.query(query).response.strip()

'No. The correct answer is 細野勇策.'

In [ ]:
question = "第2日の大会で首位に立ったのは誰でしたか？"
answer = "今平周吾"
answer = "細野勇策"
#answer = "田中裕基"
#answer = "マイケル・ヘンドリー"

query = query_template.format(question=question, answer=answer)
index.query(query).response.strip()

'YES'

In [ ]:
question = "第2日の大会で首位に立ったのは誰でしたか？"
answer = "今平周吾"
answer = "細野勇策"
answer = "田中裕基"
#answer = "マイケル・ヘンドリー"

query = query_template.format(question=question, answer=answer)
index.query(query).response.strip()

'No. 田中裕基は3位になっています。首位に立ったのは細野勇策です。'

In [ ]:
question = "第2日の大会で首位に立ったのは誰でしたか？"
answer = "今平周吾"
answer = "細野勇策"
answer = "田中裕基"
answer = "マイケル・ヘンドリー"

query = query_template.format(question=question, answer=answer)
index.query(query).response.strip()

'No. 第2日の大会で首位に立ったのは細野勇策でした。'

## 導入文の生成

いくつか生成してはよいものを選ぶのがよいだろう。

In [ ]:
query_template = """あなたは有能なクイズ作家です。次の質問と回答のペアに対して、クイズとして適切な導入文を付与してください。導入文には、質問の内容を含みません。導入文は疑問文ではありません。導入文には、解答の内容を含みません。 5つ、作成してください。

質問: {question}
回答: {answer}
"""

In [ ]:
question = "細野勇策が通算16アンダー首位となったのはどこで行われたゴルフ大会でしたか？"
answer = "東建ホームメイト・カップ"

In [ ]:
query = query_template.format(question=question, answer=answer)

query

'あなたは有能なクイズ作家です。次の質問と回答のペアに対して、クイズとして適切な導入文を付与してください。導入文には、質問の内容を含みません。導入文は疑問文ではありません。導入文には、解答の内容を含みません。 5つ、作成してください。\n\n質問: 細野勇策が通算16アンダー首位となったのはどこで行われたゴルフ大会でしたか？\n回答: 東建ホームメイト・カップ\n'

In [ ]:
index.query(query).response.strip().replace('\n','')

'1. 細野勇策が首位となったゴルフ大会はどこで行われましたか？2. 細野勇策が通算16アンダーで首位となったのはどこで行われたゴルフ大会でしょうか？3. 細野勇策が首位となったゴルフ大会の名前は何でしょうか？4. 細野勇策が首位となったゴルフ大会の場所はどこでしょうか？5. 細野勇策が通算16アンダーで首位となったゴルフ大会の名前を教えてください。'

In [ ]:
index_rel.query(query).response.strip().replace('\n','')

'1. 細野勇策がゴルフ大会で何を達成したのかご存知ですか？2. 東建ホームメイト・カップで細野勇策が何を達成したのでしょうか？3. 細野勇策がゴルフ大会で何を達成したか、ご存知ですか？4. 細野勇策がゴルフ大会で何を達成したのか、ご存知ですか？5. 東建ホームメイト・カップで細野勇策が何を達成したのか、ご存知ですか？'

# ハルシネーションのチェック

本来のハルシネーションの定義とは異なるが、ここでは扱いやすくなるよう、「出力を構成する語句のうち、インデックスやプロンプトに登場しない語句」だとする。

文全体の意味が前提知識と異なるか否かは、ここでは問題としない。そちらを問題とするならば、問題と解答と前提知識が整合するかの検証でよいだろう。


In [ ]:
question = "細野勇策が通算16アンダー首位となったのはどこで行われたゴルフ大会でしたか？"
answer = "東建ホームメイト・カップ"

In [ ]:
index_words = set([e.text for e in nlp(text).ents])
index_words

{'10アンダー',
 '11バーディー',
 '126',
 '12位',
 '12番',
 '16アンダー',
 '1ボギー',
 '1打',
 '20歳',
 '2アンダー',
 '31日',
 '3アンダー',
 '3位',
 '4打',
 '50位',
 '61',
 '64',
 '7062ヤード',
 '71',
 '79人',
 '7アンダー',
 'ニュージーランド',
 'パー71）',
 'マイケル・ヘンドリー',
 '三重県',
 '今平周吾',
 '優勝',
 '名古屋',
 '名古屋 東建ホーム',
 '多度',
 '東建多度CC',
 '田中裕基',
 '石川遼',
 '第2日',
 '細野',
 '細野勇策',
 '金谷拓実',
 '首位',
 '香妻'}

In [ ]:
question_words = set([e.text for e in nlp(question).ents])
question_words

{'16アンダー', 'ゴルフ', '細野勇策', '首位'}

In [ ]:
hullcunation_words = question_words - index_words
hullcunation_words

{'ゴルフ'}

この場合、「ゴルフ」は原文に登場しなくても、登場したほうがよい単語だろう

In [ ]:
rel_words = set()
for rel in rel_urls:
  rel_words |= set([e.text for e in nlp(rel).ents])
rel_words

{'165cm',
 '1992年4月17日',
 '1994年7月7日',
 '2010年4月4日',
 '2012年3月31日',
 'いしかわりょう',
 'ひとびと',
 'ゴルフ',
 'サマンサタバサ',
 'テレビ東京系列',
 'プロゴルファー',
 'モデル',
 '一郎',
 '国際スポーツ振興協会',
 '女子プロゴルファー',
 '日本',
 '武井玲奈',
 '石川遼',
 '石川遼スペシャル RESPECT',
 '陣',
 '香妻',
 '香妻 琴乃',
 '香妻琴乃',
 '鹿児島県鹿屋市'}

In [ ]:
hullcunation_words = question_words - (index_words | rel_words)
hullcunation_words

set()

関連情報まで含めて生成したとした場合、出力文でのみ出現する語句（固有表現）はない。

## 導入文と質問・解答をペアにしてすわりがよいかを評価
## クイズの難しさの評価
## クイズとしての面白さの評価


TBD

クイズとしての面白さには複数の軸が考えられるだろう。

In [ ]:
query_template = """あなたは有能なクイズ作家です。次の質問と回答のペアの難しさを1から5までの5段階で評価してください。簡単であれば1を、難しければ5を出力してください。


質問: {question}
回答: {answer}
"""

In [ ]:
question = "第2日の大会で首位に立ったのは誰でしたか？"
answer = "今平周吾"
answer = "細野勇策"
#answer = "田中裕基"
#answer = "マイケル・ヘンドリー"

query = query_template.format(question=question, answer=answer)
index.query(query).response.strip()

'1'

In [ ]:
query = """あなたは有能なクイズ作家です。細野勇策を解答とする、難しさ5の四択クイズを作ってください。難しさは1から5までの5段階です。簡単であれば1を、難しければ5です。
"""
index.query(query).response.strip()

'Q. 細野勇策は、東建多度CC名古屋で開催された東建ホームメイト・カップ第2日において、何スコアで首位に立ったか？\n\nA. \nA. 61\nB. 64\nC. 71\nD. 126'

In [ ]:
query = """あなたは有能なクイズ作家です。細野勇策を解答とする、難しさ1の四択クイズを作ってください。難しさは1から5までの5段階です。簡単であれば1を、難しければ5です。
"""
index.query(query).response.strip()

'Q. 細野勇策は、東建多度CC名古屋で開催された東建ホームメイト・カップ第2日で何をマークしましたか？\n\nA. \nA. 61\nB. 64\nC. 71\nD. 126'

In [ ]:
query = """あなたは有能なクイズ作家です。細野勇策を解答とする、難しさ3の四択クイズを作ってください。難しさは1から5までの5段階です。簡単であれば1を、難しければ5です。
"""
index.query(query).response.strip()

'Q. 細野勇策は、東建多度CC名古屋で開催された東建ホームメイト・カップ第2日において、何スコアで首位に立ったか？\n\nA. \nA. 61\nB. 64\nC. 71\nD. 126'

クイズの難しさについて、何か言えているわけではなさそう。いくつか例示が必要なのだろう。

あとは、マルチホップが必要な質問を作れるか？その確認のためにマルチホップQAができるか。どちらもステップバイステップのCoT(Chain of Thoughts)で可能ではないか?

以上

それらしいものを生成できるが、高品質かは別問題か。「壁打ちにはよい」とされるが、使いものにするには難易度が（人手をかける必要を含めて）高くなる印象。